# Import libraries

In [39]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
import datetime
import plotly.graph_objects as go
import plotly.express as px

# Import Dataset from fbref

In [51]:
fbref_data = {}

# Loop through the seasons from 2017-2018 to 2022-2023
for year in range(17, 24):
    season = f"20{year:02d}-20{year+1:02d}"
    url = f"https://fbref.com/en/squads/822bd0ba/{season}/Liverpool-Stats"
    fbref_data[season] = pd.read_html(url)

## Getting standard stats data for Liverpool FC from EPL Season 2017-2018 to 2023-2024

In [ ]:
for season in fbref_data.keys():
    # Construct the variable name based on the naming convention
    variable_name = f"standard_stats_season_{season.replace('-', '_')}"

    # Assign the DataFrame to the dynamically created variable
    # Using the first table from each season's data
    globals()[variable_name] = fbref_data[season][0]

# Loop over the seasons
for year in range(17, 24):
    # Construct the variable name
    df_name = f"standard_stats_season_20{year:02d}_20{year+1:02d}"

    # Access the DataFrame using globals()
    df = globals()[df_name]

    # Perform the operations
    # 1. Combining multi-level column names
    df.columns = [' '.join(col).strip() for col in df.columns]

    # 2. Resetting the index
    df = df.reset_index(drop=True)

    # 3. Renaming columns to handle 'level_0' occurrences
    new_columns = []
    for col in df.columns:
        if 'level_0' in col:
            new_col = col.split()[-1]  # takes the last name
        else:
            new_col = col
        new_columns.append(new_col)
    df.columns = new_columns

    # 4. Filling NaN values with 0
    df = df.fillna(0)

    # 5. Optionally printing the head (first few rows) of each DataFrame
    print(f"Head of DataFrame for {df_name}:")
    print(df.head())

    # Update the DataFrame in globals()
    globals()[df_name] = df

Head of DataFrame for standard_stats_season_2017_2018:
                Player   Nation Pos   Age  MP  Playing Time Starts  \
0        Mohamed Salah   eg EGY  FW  25.0  36                   34   
1      Roberto Firmino   br BRA  FW  25.0  37                   32   
2           Sadio Mané   sn SEN  FW  25.0  29                   28   
3  Georginio Wijnaldum   nl NED  MF  26.0  33                   27   
4     Jordan Henderson  eng ENG  MF  27.0  27                   25   

   Playing Time Min  Playing Time 90s  Performance Gls  Performance Ast  ...  \
0            2907.0              32.3             32.0             10.0  ...   
1            2763.0              30.7             15.0              7.0  ...   
2            2190.0              24.3             10.0              7.0  ...   
3            2387.0              26.5              1.0              2.0  ...   
4            2171.0              24.1              1.0              1.0  ...   

   Per 90 Minutes Ast  Per 90 Minutes G+A  

In [ ]:
# Loop over the seasons and print the shape of each DataFrame
for year in range(17, 24):
    # Construct the variable name based on the naming convention
    df_name = f"standard_stats_season_20{year:02d}_20{year+1:02d}"

    # Access the DataFrame using globals() and print its shape
    if df_name in globals():
        print(f"{df_name}: {globals()[df_name].shape}")
    else:
        print(f"DataFrame {df_name} not found.")

standard_stats_season_2017_2018: (34, 34)
standard_stats_season_2018_2019: (28, 34)
standard_stats_season_2019_2020: (31, 34)
standard_stats_season_2020_2021: (37, 34)
standard_stats_season_2021_2022: (37, 34)
standard_stats_season_2022_2023: (37, 34)
standard_stats_season_2023_2024: (40, 34)


In [ ]:
# Assuming we use the most recent season (2023-2024) as the standard for column order
standard_column_order = globals().get('standard_stats_season_2023_2024', pd.DataFrame()).columns.tolist()

# Initialize a list to store the modified DataFrames
all_seasons_data = []

# Loop over the seasons again to align all DataFrames and add the 'season' column
for year in range(17, 24):
    # Construct the variable name
    df_name = f"standard_stats_season_20{year:02d}_20{year+1:02d}"

    # Access the DataFrame using globals()
    if df_name in globals():
        # Create a copy to avoid modifying the original DataFrame
        df = globals()[df_name].copy()

        # Align the DataFrame to the standard column order, filling missing columns with NaN
        df = df.reindex(columns=standard_column_order, fill_value=np.nan)

        # Add the season identifier column
        df['season'] = f"20{year:02d}-20{year+1:02d}"

        # Append the modified DataFrame to the list
        all_seasons_data.append(df)

# Concatenate all DataFrames
standard_stats_lfc = pd.concat(all_seasons_data, ignore_index=True)

In [ ]:
standard_stats_lfc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 244 entries, 0 to 243
Data columns (total 35 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Player                   244 non-null    object 
 1   Nation                   244 non-null    object 
 2   Pos                      244 non-null    object 
 3   Age                      244 non-null    object 
 4   Playing Time MP          40 non-null     float64
 5   Playing Time Starts      244 non-null    int64  
 6   Playing Time Min         244 non-null    float64
 7   Playing Time 90s         244 non-null    float64
 8   Performance Gls          244 non-null    float64
 9   Performance Ast          244 non-null    float64
 10  Performance G+A          244 non-null    float64
 11  Performance G-PK         244 non-null    float64
 12  Performance PK           244 non-null    float64
 13  Performance PKatt        244 non-null    float64
 14  Performance CrdY         2

In [ ]:
standard_stats_lfc['Player'].value_counts()

Player
Mohamed Salah             7
Joël Matip                7
Curtis Jones              7
Squad Total               7
Trent Alexander-Arnold    7
                         ..
Jake Cain                 1
Ben Davies                1
Liam Hughes               1
Billy Koumetio            1
Calum Scanlon             1
Name: count, Length: 88, dtype: int64

In [ ]:
standard_stats_lfc = standard_stats_lfc[(standard_stats_lfc['Player'] != 'Squad Total') & (standard_stats_lfc['Player'] != 'Opponent Total')]

In [ ]:
standard_stats_lfc.info()

<class 'pandas.core.frame.DataFrame'>
Index: 230 entries, 0 to 241
Data columns (total 35 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Player                   230 non-null    object 
 1   Nation                   230 non-null    object 
 2   Pos                      230 non-null    object 
 3   Age                      230 non-null    object 
 4   Playing Time MP          38 non-null     float64
 5   Playing Time Starts      230 non-null    int64  
 6   Playing Time Min         230 non-null    float64
 7   Playing Time 90s         230 non-null    float64
 8   Performance Gls          230 non-null    float64
 9   Performance Ast          230 non-null    float64
 10  Performance G+A          230 non-null    float64
 11  Performance G-PK         230 non-null    float64
 12  Performance PK           230 non-null    float64
 13  Performance PKatt        230 non-null    float64
 14  Performance CrdY         230 no

In [ ]:
standard_stats_lfc.to_csv('standard_stats_lfc.csv', sep=',', index=False, encoding='utf-8-sig')

## Getting scores and fixtures data for Liverpool FC from Season 2017-2018 to 2023-2024

In [ ]:
for season in fbref_data.keys():
    # Construct the variable name based on the naming convention
    variable_name = f"match_season_{season.replace('-', '_')}"

    # Assign the DataFrame to the dynamically created variable
    # Using the first table from each season's data
    globals()[variable_name] = fbref_data[season][1]

# Loop over the seasons
for year in range(17, 24):
    # Construct the variable name
    df_name = f"match_season_20{year:02d}_20{year+1:02d}"

    # Access the DataFrame using globals()
    df = globals()[df_name]

    # Perform the operations
    # 1. Combining multi-level column names
    df.columns = [col.strip().replace(' ', '') for col in df.columns]

    # 2. Resetting the index
    df = df.reset_index(drop=True)

    # 3. Renaming columns to handle 'level_0' occurrences
    new_columns = []
    for col in df.columns:
        if 'level_0' in col:
            new_col = col.split()[-1]  # takes the last name
        else:
            new_col = col
        new_columns.append(new_col)
    df.columns = new_columns

    # 4. Filling NaN values with 0
    df = df.fillna(0)

    # 5. Optionally printing the head (first few rows) of each DataFrame
    print(f"Head of DataFrame for {df_name}:")
    print(df.head())

    # Update the DataFrame in globals()
    globals()[df_name] = df

Head of DataFrame for match_season_2017_2018:
         Date   Time            Comp           Round  Day Venue Result  GF  \
0  2017-08-12  12:30  Premier League     Matchweek 1  Sat  Away      D   3   
1  2017-08-15  20:45    Champions Lg  Play-off round  Tue  Away      W   2   
2  2017-08-19  15:00  Premier League     Matchweek 2  Sat  Home      W   1   
3  2017-08-23  19:45    Champions Lg  Play-off round  Wed  Home      W   4   
4  2017-08-27  16:00  Premier League     Matchweek 3  Sun  Home      W   4   

   GA        Opponent   xG  xGA  Poss  Attendance           Captain Formation  \
0   3         Watford  2.6  2.1    54       20407  Jordan Henderson     4-3-3   
1   1   de Hoffenheim  0.0  0.0    37       25568  Jordan Henderson     4-3-3   
2   0  Crystal Palace  2.5  0.7    71       53138  Jordan Henderson     4-3-3   
3   2   de Hoffenheim  0.0  0.0    45       51808  Jordan Henderson     4-3-3   
4   0         Arsenal  3.1  0.6    48       53206  Jordan Henderson     4-3-3   

In [ ]:
# Loop over the seasons and print the shape of each DataFrame
for year in range(17, 24):
    # Construct the variable name based on the naming convention
    df_name = f"match_season_20{year:02d}_20{year+1:02d}"

    # Access the DataFrame using globals() and print its shape
    if df_name in globals():
        print(f"{df_name}: {globals()[df_name].shape}")
    else:
        print(f"DataFrame {df_name} not found.")

match_season_2017_2018: (56, 19)
match_season_2018_2019: (53, 19)
match_season_2019_2020: (55, 19)
match_season_2020_2021: (53, 19)
match_season_2021_2022: (63, 19)
match_season_2022_2023: (52, 19)
match_season_2023_2024: (58, 19)


In [ ]:
# Assuming we use the most recent season (2023-2024) as the standard for column order
standard_column_order = globals().get('match_season_2023_2024', pd.DataFrame()).columns.tolist()

# Initialize a list to store the modified DataFrames
all_seasons_data = []

# Loop over the seasons again to align all DataFrames and add the 'season' column
for year in range(17, 24):
    # Construct the variable name
    df_name = f"match_season_20{year:02d}_20{year+1:02d}"

    # Access the DataFrame using globals()
    if df_name in globals():
        # Create a copy to avoid modifying the original DataFrame
        df = globals()[df_name].copy()

        # Align the DataFrame to the standard column order, filling missing columns with NaN
        df = df.reindex(columns=standard_column_order, fill_value=np.nan)

        # Add the season identifier column
        df['season'] = f"20{year:02d}-20{year+1:02d}"

        # Append the modified DataFrame to the list
        all_seasons_data.append(df)

# Concatenate all DataFrames
match_lfc = pd.concat(all_seasons_data, ignore_index=True)

In [ ]:
match_lfc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 390 entries, 0 to 389
Data columns (total 20 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Date         390 non-null    object 
 1   Time         390 non-null    object 
 2   Comp         390 non-null    object 
 3   Round        390 non-null    object 
 4   Day          390 non-null    object 
 5   Venue        390 non-null    object 
 6   Result       390 non-null    object 
 7   GF           390 non-null    object 
 8   GA           390 non-null    object 
 9   Opponent     390 non-null    object 
 10  xG           390 non-null    float64
 11  xGA          390 non-null    float64
 12  Poss         390 non-null    float64
 13  Attendance   390 non-null    float64
 14  Captain      390 non-null    object 
 15  Formation    390 non-null    object 
 16  Referee      390 non-null    object 
 17  MatchReport  390 non-null    object 
 18  Notes        390 non-null    object 
 19  season  

In [ ]:
match_lfc.to_csv('match_lfc.csv', sep=',', index=False, encoding='utf-8-sig')

## Getting goalkeeper data for Liverpool FC from Season 2017-2018 to 2023-2024

In [ ]:
# Loop over the seasons and print the shape of each DataFrame
for year in range(17, 24):
    # Construct the variable name based on the naming convention
    df_name = f"match_season_20{year:02d}_20{year+1:02d}"

    # Access the DataFrame using globals() and print its shape
    if df_name in globals():
        print(f"{df_name}: {globals()[df_name].shape}")
    else:
        print(f"DataFrame {df_name} not found.")

match_season_2017_2018: (56, 19)
match_season_2018_2019: (53, 19)
match_season_2019_2020: (55, 19)
match_season_2020_2021: (53, 19)
match_season_2021_2022: (63, 19)
match_season_2022_2023: (52, 19)
match_season_2023_2024: (58, 19)


In [ ]:
# Assuming we use the most recent season (2023-2024) as the standard for column order
standard_column_order = globals().get('match_season_2023_2024', pd.DataFrame()).columns.tolist()

# Initialize a list to store the modified DataFrames
all_seasons_data = []

# Loop over the seasons again to align all DataFrames and add the 'season' column
for year in range(17, 24):
    # Construct the variable name
    df_name = f"match_season_20{year:02d}_20{year+1:02d}"

    # Access the DataFrame using globals()
    if df_name in globals():
        # Create a copy to avoid modifying the original DataFrame
        df = globals()[df_name].copy()

        # Align the DataFrame to the standard column order, filling missing columns with NaN
        df = df.reindex(columns=standard_column_order, fill_value=np.nan)

        # Add the season identifier column
        df['season'] = f"20{year:02d}-20{year+1:02d}"

        # Append the modified DataFrame to the list
        all_seasons_data.append(df)

# Concatenate all DataFrames
match_lfc = pd.concat(all_seasons_data, ignore_index=True)

In [ ]:
match_lfc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 390 entries, 0 to 389
Data columns (total 20 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Date         390 non-null    object 
 1   Time         390 non-null    object 
 2   Comp         390 non-null    object 
 3   Round        390 non-null    object 
 4   Day          390 non-null    object 
 5   Venue        390 non-null    object 
 6   Result       390 non-null    object 
 7   GF           390 non-null    object 
 8   GA           390 non-null    object 
 9   Opponent     390 non-null    object 
 10  xG           390 non-null    float64
 11  xGA          390 non-null    float64
 12  Poss         390 non-null    float64
 13  Attendance   390 non-null    float64
 14  Captain      390 non-null    object 
 15  Formation    390 non-null    object 
 16  Referee      390 non-null    object 
 17  MatchReport  390 non-null    object 
 18  Notes        390 non-null    object 
 19  season  

In [ ]:
match_lfc.to_csv('match_lfc.csv', sep=',', index=False, encoding='utf-8-sig')

## Getting shooting data for Liverpool FC from EPL Season 2017-2018 to 2023-2024

In [ ]:
for season in fbref_data.keys():
    # Construct the variable name based on the naming convention
    variable_name = f"shooting_season_{season.replace('-', '_')}"

    # Assign the DataFrame to the dynamically created variable
    # Using the first table from each season's data
    globals()[variable_name] = fbref_data[season][4]

# Loop over the seasons
for year in range(17, 24):
    # Construct the variable name
    df_name = f"shooting_season_20{year:02d}_20{year+1:02d}"

    # Access the DataFrame using globals()
    df = globals()[df_name]

    # Perform the operations
    # 1. Combining multi-level column names
    df.columns = [' '.join(col).strip() for col in df.columns]

    # 2. Resetting the index
    df = df.reset_index(drop=True)

    # 3. Renaming columns to handle 'level_0' occurrences
    new_columns = []
    for col in df.columns:
        if 'level_0' in col:
            new_col = col.split()[-1]  # takes the last name
        else:
            new_col = col
        new_columns.append(new_col)
    df.columns = new_columns

    # 4. Filling NaN values with 0
    df = df.fillna(0)

    # 5. Optionally printing the head (first few rows) of each DataFrame
    print(f"Head of DataFrame for {df_name}:")
    print(df.head())

    # Update the DataFrame in globals()
    globals()[df_name] = df

Head of DataFrame for shooting_season_2017_2018:
                Player   Nation Pos   Age   90s  Standard Gls  Standard Sh  \
0        Mohamed Salah   eg EGY  FW  25.0  32.3            32          142   
1      Roberto Firmino   br BRA  FW  25.0  30.7            15           83   
2           Sadio Mané   sn SEN  FW  25.0  24.3            10           70   
3  Georginio Wijnaldum   nl NED  MF  26.0  26.5             1           38   
4     Jordan Henderson  eng ENG  MF  27.0  24.1             1           15   

   Standard SoT  Standard SoT%  Standard Sh/90  ...  Standard Dist  \
0            64           45.1            4.40  ...           15.3   
1            36           43.4            2.70  ...           17.1   
2            26           37.1            2.88  ...           15.4   
3             9           23.7            1.43  ...           19.0   
4             3           20.0            0.62  ...           23.8   

   Standard FK  Standard PK  Standard PKatt  Expected xG  Exp

In [ ]:
# Loop over the seasons and print the shape of each DataFrame
for year in range(17, 24):
    # Construct the variable name based on the naming convention
    df_name = f"shooting_season_20{year:02d}_20{year+1:02d}"

    # Access the DataFrame using globals() and print its shape
    if df_name in globals():
        print(f"{df_name}: {globals()[df_name].shape}")
    else:
        print(f"DataFrame {df_name} not found.")

shooting_season_2017_2018: (29, 23)
shooting_season_2018_2019: (25, 23)
shooting_season_2019_2020: (26, 23)
shooting_season_2020_2021: (30, 23)
shooting_season_2021_2022: (29, 23)
shooting_season_2022_2023: (30, 23)
shooting_season_2023_2024: (31, 23)


In [ ]:
# Assuming we use the most recent season (2023-2024) as the standard for column order
standard_column_order = globals().get('shooting_season_2023_2024', pd.DataFrame()).columns.tolist()

# Initialize a list to store the modified DataFrames
all_seasons_data = []

# Loop over the seasons again to align all DataFrames and add the 'season' column
for year in range(17, 24):
    # Construct the variable name
    df_name = f"shooting_season_20{year:02d}_20{year+1:02d}"

    # Access the DataFrame using globals()
    if df_name in globals():
        # Create a copy to avoid modifying the original DataFrame
        df = globals()[df_name].copy()

        # Align the DataFrame to the standard column order, filling missing columns with NaN
        df = df.reindex(columns=standard_column_order, fill_value=np.nan)

        # Add the season identifier column
        df['season'] = f"20{year:02d}-20{year+1:02d}"

        # Append the modified DataFrame to the list
        all_seasons_data.append(df)

# Concatenate all DataFrames
shooting_lfc = pd.concat(all_seasons_data, ignore_index=True)

In [ ]:
shooting_lfc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 24 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Player            200 non-null    object 
 1   Nation            200 non-null    object 
 2   Pos               200 non-null    object 
 3   Age               200 non-null    object 
 4   90s               200 non-null    float64
 5   Standard Gls      200 non-null    int64  
 6   Standard Sh       200 non-null    int64  
 7   Standard SoT      200 non-null    int64  
 8   Standard SoT%     200 non-null    float64
 9   Standard Sh/90    200 non-null    float64
 10  Standard SoT/90   200 non-null    float64
 11  Standard G/Sh     200 non-null    float64
 12  Standard G/SoT    200 non-null    float64
 13  Standard Dist     200 non-null    float64
 14  Standard FK       200 non-null    int64  
 15  Standard PK       200 non-null    int64  
 16  Standard PKatt    200 non-null    int64  
 1

In [ ]:
shooting_lfc['Player'].value_counts()

Player
Mohamed Salah             7
Joël Matip                7
Virgil van Dijk           7
Opponent Total            7
Trent Alexander-Arnold    7
                         ..
Philippe Coutinho         1
Dominic Solanke           1
Danny Ings                1
Rafael Camacho            1
James McConnell           1
Name: count, Length: 65, dtype: int64

In [ ]:
shooting_lfc = shooting_lfc[(shooting_lfc['Player'] != 'Squad Total') & (shooting_lfc['Player'] != 'Opponent Total')]

In [ ]:
shooting_lfc.info()

<class 'pandas.core.frame.DataFrame'>
Index: 186 entries, 0 to 197
Data columns (total 24 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Player            186 non-null    object 
 1   Nation            186 non-null    object 
 2   Pos               186 non-null    object 
 3   Age               186 non-null    object 
 4   90s               186 non-null    float64
 5   Standard Gls      186 non-null    int64  
 6   Standard Sh       186 non-null    int64  
 7   Standard SoT      186 non-null    int64  
 8   Standard SoT%     186 non-null    float64
 9   Standard Sh/90    186 non-null    float64
 10  Standard SoT/90   186 non-null    float64
 11  Standard G/Sh     186 non-null    float64
 12  Standard G/SoT    186 non-null    float64
 13  Standard Dist     186 non-null    float64
 14  Standard FK       186 non-null    int64  
 15  Standard PK       186 non-null    int64  
 16  Standard PKatt    186 non-null    int64  
 17  Ex

In [ ]:
shooting_lfc.to_csv('shooting_lfc.csv', sep=',', index=False, encoding='utf-8-sig')

## Getting passing data for Liverpool FC from EPL Season 2017-2018 to 2023-2024

In [ ]:
for season in fbref_data.keys():
    # Construct the variable name based on the naming convention
    variable_name = f"passing_season_{season.replace('-', '_')}"

    # Assign the DataFrame to the dynamically created variable
    # Using the first table from each season's data
    globals()[variable_name] = fbref_data[season][5]

# Loop over the seasons
for year in range(17, 24):
    # Construct the variable name
    df_name = f"passing_season_20{year:02d}_20{year+1:02d}"

    # Access the DataFrame using globals()
    df = globals()[df_name]

    # Perform the operations
    # 1. Combining multi-level column names
    df.columns = [' '.join(col).strip() for col in df.columns]

    # 2. Resetting the index
    df = df.reset_index(drop=True)

    # 3. Renaming columns to handle 'level_0' occurrences
    new_columns = []
    for col in df.columns:
        if 'level_0' in col:
            new_col = col.split()[-1]  # takes the last name
        else:
            new_col = col
        new_columns.append(new_col)
    df.columns = new_columns

    # 4. Filling NaN values with 0
    df = df.fillna(0)

    # 5. Optionally printing the head (first few rows) of each DataFrame
    print(f"Head of DataFrame for {df_name}:")
    print(df.head())

    # Update the DataFrame in globals()
    globals()[df_name] = df

Head of DataFrame for passing_season_2017_2018:
                Player   Nation Pos   Age   90s  Total Cmp  Total Att  \
0        Mohamed Salah   eg EGY  FW  25.0  32.3        772       1055   
1      Roberto Firmino   br BRA  FW  25.0  30.7        869       1233   
2           Sadio Mané   sn SEN  FW  25.0  24.3        787       1005   
3  Georginio Wijnaldum   nl NED  MF  26.0  26.5       1145       1283   
4     Jordan Henderson  eng ENG  MF  27.0  24.1       1769       2117   

   Total Cmp%  Total TotDist  Total PrgDist  ...  Ast  xAG  Expected xA  \
0        73.2          11021           3263  ...   10  7.5          6.1   
1        70.5          11802           3204  ...    7  6.5          5.6   
2        78.3          11585           2403  ...    7  5.4          4.9   
3        89.2          18447           3622  ...    2  1.2          1.6   
4        83.6          33580           9305  ...    1  1.8          2.4   

   Expected A-xAG  KP  1/3  PPA  CrsPA  PrgP  Matches  
0     

In [ ]:
# Loop over the seasons and print the shape of each DataFrame
for year in range(17, 24):
    # Construct the variable name based on the naming convention
    df_name = f"passing_season_20{year:02d}_20{year+1:02d}"

    # Access the DataFrame using globals() and print its shape
    if df_name in globals():
        print(f"{df_name}: {globals()[df_name].shape}")
    else:
        print(f"DataFrame {df_name} not found.")

passing_season_2017_2018: (29, 29)
passing_season_2018_2019: (25, 29)
passing_season_2019_2020: (26, 29)
passing_season_2020_2021: (30, 29)
passing_season_2021_2022: (29, 29)
passing_season_2022_2023: (30, 29)
passing_season_2023_2024: (31, 29)


In [ ]:
# Assuming we use the most recent season (2023-2024) as the standard for column order
standard_column_order = globals().get('passing_season_2023_2024', pd.DataFrame()).columns.tolist()

# Initialize a list to store the modified DataFrames
all_seasons_data = []

# Loop over the seasons again to align all DataFrames and add the 'season' column
for year in range(17, 24):
    # Construct the variable name
    df_name = f"passing_season_20{year:02d}_20{year+1:02d}"

    # Access the DataFrame using globals()
    if df_name in globals():
        # Create a copy to avoid modifying the original DataFrame
        df = globals()[df_name].copy()

        # Align the DataFrame to the standard column order, filling missing columns with NaN
        df = df.reindex(columns=standard_column_order, fill_value=np.nan)

        # Add the season identifier column
        df['season'] = f"20{year:02d}-20{year+1:02d}"

        # Append the modified DataFrame to the list
        all_seasons_data.append(df)

# Concatenate all DataFrames
passing_lfc = pd.concat(all_seasons_data, ignore_index=True)

In [ ]:
passing_lfc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 30 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Player          200 non-null    object 
 1   Nation          200 non-null    object 
 2   Pos             200 non-null    object 
 3   Age             200 non-null    object 
 4   90s             200 non-null    float64
 5   Total Cmp       200 non-null    int64  
 6   Total Att       200 non-null    int64  
 7   Total Cmp%      200 non-null    float64
 8   Total TotDist   200 non-null    int64  
 9   Total PrgDist   200 non-null    int64  
 10  Short Cmp       200 non-null    int64  
 11  Short Att       200 non-null    int64  
 12  Short Cmp%      200 non-null    float64
 13  Medium Cmp      200 non-null    int64  
 14  Medium Att      200 non-null    int64  
 15  Medium Cmp%     200 non-null    float64
 16  Long Cmp        200 non-null    int64  
 17  Long Att        200 non-null    int

In [ ]:
passing_lfc['Player'].value_counts()

Player
Mohamed Salah             7
Joël Matip                7
Virgil van Dijk           7
Opponent Total            7
Trent Alexander-Arnold    7
                         ..
Philippe Coutinho         1
Dominic Solanke           1
Danny Ings                1
Rafael Camacho            1
James McConnell           1
Name: count, Length: 65, dtype: int64

In [ ]:
passing_lfc = passing_lfc[(passing_lfc['Player'] != 'Squad Total') & (passing_lfc['Player'] != 'Opponent Total')]

In [ ]:
passing_lfc.info()

<class 'pandas.core.frame.DataFrame'>
Index: 186 entries, 0 to 197
Data columns (total 30 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Player          186 non-null    object 
 1   Nation          186 non-null    object 
 2   Pos             186 non-null    object 
 3   Age             186 non-null    object 
 4   90s             186 non-null    float64
 5   Total Cmp       186 non-null    int64  
 6   Total Att       186 non-null    int64  
 7   Total Cmp%      186 non-null    float64
 8   Total TotDist   186 non-null    int64  
 9   Total PrgDist   186 non-null    int64  
 10  Short Cmp       186 non-null    int64  
 11  Short Att       186 non-null    int64  
 12  Short Cmp%      186 non-null    float64
 13  Medium Cmp      186 non-null    int64  
 14  Medium Att      186 non-null    int64  
 15  Medium Cmp%     186 non-null    float64
 16  Long Cmp        186 non-null    int64  
 17  Long Att        186 non-null    int64  


In [ ]:
passing_lfc.to_csv('passing_lfc.csv', sep=',', index=False, encoding='utf-8-sig')

## Getting passing type data for Liverpool FC from EPL Season 2017-2018 to 2023-2024

In [41]:
for season in fbref_data.keys():
    # Construct the variable name based on the naming convention
    variable_name = f"pass_type_season_{season.replace('-', '_')}"

    # Assign the DataFrame to the dynamically created variable
    # Using the first table from each season's data
    globals()[variable_name] = fbref_data[season][6]

# Loop over the seasons
for year in range(17, 24):
    # Construct the variable name
    df_name = f"pass_type_season_20{year:02d}_20{year+1:02d}"

    # Access the DataFrame using globals()
    df = globals()[df_name]

    # Perform the operations
    # 1. Combining multi-level column names
    df.columns = [' '.join(col).strip() for col in df.columns]

    # 2. Resetting the index
    df = df.reset_index(drop=True)

    # 3. Renaming columns to handle 'level_0' occurrences
    new_columns = []
    for col in df.columns:
        if 'level_0' in col:
            new_col = col.split()[-1]  # takes the last name
        else:
            new_col = col
        new_columns.append(new_col)
    df.columns = new_columns

    # 4. Filling NaN values with 0
    df = df.fillna(0)

    # 5. Optionally printing the head (first few rows) of each DataFrame
    print(f"Head of DataFrame for {df_name}:")
    print(df.head())

    # Update the DataFrame in globals()
    globals()[df_name] = df

Head of DataFrame for pass_type_season_2017_2018:
                Player   Nation Pos   Age   90s   Att  Pass Types Live  \
0        Mohamed Salah   eg EGY  FW  25.0  32.3  1055              955   
1      Roberto Firmino   br BRA  FW  25.0  30.7  1233             1180   
2           Sadio Mané   sn SEN  FW  25.0  24.3  1005              991   
3  Georginio Wijnaldum   nl NED  MF  26.0  26.5  1283             1245   
4     Jordan Henderson  eng ENG  MF  27.0  24.1  2117             2061   

   Pass Types Dead  Pass Types FK  Pass Types TB  ...  Pass Types Crs  \
0               96             12              8  ...              86   
1               48              5             17  ...              43   
2               11              2             10  ...              32   
3               29              8              2  ...               2   
4               53             44              7  ...              25   

   Pass Types TI  Pass Types CK  Corner Kicks In  Corner Kicks Out

In [42]:
# Loop over the seasons and print the shape of each DataFrame
for year in range(17, 24):
    # Construct the variable name based on the naming convention
    df_name = f"pass_type_season_20{year:02d}_20{year+1:02d}"

    # Access the DataFrame using globals() and print its shape
    if df_name in globals():
        print(f"{df_name}: {globals()[df_name].shape}")
    else:
        print(f"DataFrame {df_name} not found.")

pass_type_season_2017_2018: (29, 21)
pass_type_season_2018_2019: (25, 21)
pass_type_season_2019_2020: (26, 21)
pass_type_season_2020_2021: (30, 21)
pass_type_season_2021_2022: (29, 21)
pass_type_season_2022_2023: (30, 21)
pass_type_season_2023_2024: (31, 21)


In [43]:
# Assuming we use the most recent season (2023-2024) as the standard for column order
standard_column_order = globals().get('pass_type_season_2023_2024', pd.DataFrame()).columns.tolist()

# Initialize a list to store the modified DataFrames
all_seasons_data = []

# Loop over the seasons again to align all DataFrames and add the 'season' column
for year in range(17, 24):
    # Construct the variable name
    df_name = f"pass_type_season_20{year:02d}_20{year+1:02d}"

    # Access the DataFrame using globals()
    if df_name in globals():
        # Create a copy to avoid modifying the original DataFrame
        df = globals()[df_name].copy()

        # Align the DataFrame to the standard column order, filling missing columns with NaN
        df = df.reindex(columns=standard_column_order, fill_value=np.nan)

        # Add the season identifier column
        df['season'] = f"20{year:02d}-20{year+1:02d}"

        # Append the modified DataFrame to the list
        all_seasons_data.append(df)

# Concatenate all DataFrames
pass_type_lfc = pd.concat(all_seasons_data, ignore_index=True)

In [44]:
pass_type_lfc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 22 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Player            200 non-null    object 
 1   Nation            200 non-null    object 
 2   Pos               200 non-null    object 
 3   Age               200 non-null    object 
 4   90s               200 non-null    float64
 5   Att               200 non-null    int64  
 6   Pass Types Live   200 non-null    int64  
 7   Pass Types Dead   200 non-null    int64  
 8   Pass Types FK     200 non-null    int64  
 9   Pass Types TB     200 non-null    int64  
 10  Pass Types Sw     200 non-null    int64  
 11  Pass Types Crs    200 non-null    int64  
 12  Pass Types TI     200 non-null    int64  
 13  Pass Types CK     200 non-null    int64  
 14  Corner Kicks In   200 non-null    int64  
 15  Corner Kicks Out  200 non-null    int64  
 16  Corner Kicks Str  200 non-null    int64  
 1

In [45]:
pass_type_lfc['Player'].value_counts()

Player
Mohamed Salah             7
Joël Matip                7
Virgil van Dijk           7
Opponent Total            7
Trent Alexander-Arnold    7
                         ..
Philippe Coutinho         1
Dominic Solanke           1
Danny Ings                1
Rafael Camacho            1
James McConnell           1
Name: count, Length: 65, dtype: int64

In [46]:
pass_type_lfc = pass_type_lfc[(pass_type_lfc['Player'] != 'Squad Total') & (pass_type_lfc['Player'] != 'Opponent Total')]

In [47]:
pass_type_lfc.info()

<class 'pandas.core.frame.DataFrame'>
Index: 186 entries, 0 to 197
Data columns (total 22 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Player            186 non-null    object 
 1   Nation            186 non-null    object 
 2   Pos               186 non-null    object 
 3   Age               186 non-null    object 
 4   90s               186 non-null    float64
 5   Att               186 non-null    int64  
 6   Pass Types Live   186 non-null    int64  
 7   Pass Types Dead   186 non-null    int64  
 8   Pass Types FK     186 non-null    int64  
 9   Pass Types TB     186 non-null    int64  
 10  Pass Types Sw     186 non-null    int64  
 11  Pass Types Crs    186 non-null    int64  
 12  Pass Types TI     186 non-null    int64  
 13  Pass Types CK     186 non-null    int64  
 14  Corner Kicks In   186 non-null    int64  
 15  Corner Kicks Out  186 non-null    int64  
 16  Corner Kicks Str  186 non-null    int64  
 17  Ou

In [49]:
pass_type_lfc.to_csv('pass_type_lfc.csv', sep=',', index=False, encoding='utf-8-sig')

## Getting goal and shot creation data for Liverpool FC from EPL Season 2017-2018 to 2023-2024

In [ ]:
for season in fbref_data.keys():
    # Construct the variable name based on the naming convention
    variable_name = f"gsc_season_{season.replace('-', '_')}"

    # Assign the DataFrame to the dynamically created variable
    # Using the first table from each season's data
    globals()[variable_name] = fbref_data[season][7]

# Loop over the seasons
for year in range(17, 24):
    # Construct the variable name
    df_name = f"gsc_season_20{year:02d}_20{year+1:02d}"

    # Access the DataFrame using globals()
    df = globals()[df_name]

    # Perform the operations
    # 1. Combining multi-level column names
    df.columns = [' '.join(col).strip() for col in df.columns]

    # 2. Resetting the index
    df = df.reset_index(drop=True)

    # 3. Renaming columns to handle 'level_0' occurrences
    new_columns = []
    for col in df.columns:
        if 'level_0' in col:
            new_col = col.split()[-1]  # takes the last name
        else:
            new_col = col
        new_columns.append(new_col)
    df.columns = new_columns

    # 4. Filling NaN values with 0
    df = df.fillna(0)

    # 5. Optionally printing the head (first few rows) of each DataFrame
    print(f"Head of DataFrame for {df_name}:")
    print(df.head())

    # Update the DataFrame in globals()
    globals()[df_name] = df

In [ ]:
# Loop over the seasons and print the shape of each DataFrame
for year in range(17, 24):
    # Construct the variable name based on the naming convention
    df_name = f"gsc_season_20{year:02d}_20{year+1:02d}"

    # Access the DataFrame using globals() and print its shape
    if df_name in globals():
        print(f"{df_name}: {globals()[df_name].shape}")
    else:
        print(f"DataFrame {df_name} not found.")

In [ ]:
# Assuming we use the most recent season (2023-2024) as the standard for column order
standard_column_order = globals().get('gsc_season_2023_2024', pd.DataFrame()).columns.tolist()

# Initialize a list to store the modified DataFrames
all_seasons_data = []

# Loop over the seasons again to align all DataFrames and add the 'season' column
for year in range(17, 24):
    # Construct the variable name
    df_name = f"gsc_season_20{year:02d}_20{year+1:02d}"

    # Access the DataFrame using globals()
    if df_name in globals():
        # Create a copy to avoid modifying the original DataFrame
        df = globals()[df_name].copy()

        # Align the DataFrame to the standard column order, filling missing columns with NaN
        df = df.reindex(columns=standard_column_order, fill_value=np.nan)

        # Add the season identifier column
        df['season'] = f"20{year:02d}-20{year+1:02d}"

        # Append the modified DataFrame to the list
        all_seasons_data.append(df)

# Concatenate all DataFrames
gsc_lfc = pd.concat(all_seasons_data, ignore_index=True)

In [ ]:
gsc_lfc.info()

In [ ]:
gsc_lfc['Player'].value_counts()

In [ ]:
gsc_lfc = gsc_lfc[(gsc_lfc['Player'] != 'Squad Total') & (gsc_lfc['Player'] != 'Opponent Total')]

In [ ]:
gsc_lfc.info()

In [ ]:
gsc_lfc.to_csv('gsc_lfc.csv', sep=',', index=False, encoding='utf-8-sig')

## Getting defense data for Liverpool FC from EPL Season 2017-2018 to 2023-2024

In [ ]:
for season in fbref_data.keys():
    # Construct the variable name based on the naming convention
    variable_name = f"defense_season_{season.replace('-', '_')}"

    # Assign the DataFrame to the dynamically created variable
    # Using the first table from each season's data
    globals()[variable_name] = fbref_data[season][8]

# Loop over the seasons
for year in range(17, 24):
    # Construct the variable name
    df_name = f"defense_season_20{year:02d}_20{year+1:02d}"

    # Access the DataFrame using globals()
    df = globals()[df_name]

    # Perform the operations
    # 1. Combining multi-level column names
    df.columns = [' '.join(col).strip() for col in df.columns]

    # 2. Resetting the index
    df = df.reset_index(drop=True)

    # 3. Renaming columns to handle 'level_0' occurrences
    new_columns = []
    for col in df.columns:
        if 'level_0' in col:
            new_col = col.split()[-1]  # takes the last name
        else:
            new_col = col
        new_columns.append(new_col)
    df.columns = new_columns

    # 4. Filling NaN values with 0
    df = df.fillna(0)

    # 5. Optionally printing the head (first few rows) of each DataFrame
    print(f"Head of DataFrame for {df_name}:")
    print(df.head())

    # Update the DataFrame in globals()
    globals()[df_name] = df

In [ ]:
# Loop over the seasons and print the shape of each DataFrame
for year in range(17, 24):
    # Construct the variable name based on the naming convention
    df_name = f"defense_season_20{year:02d}_20{year+1:02d}"

    # Access the DataFrame using globals() and print its shape
    if df_name in globals():
        print(f"{df_name}: {globals()[df_name].shape}")
    else:
        print(f"DataFrame {df_name} not found.")

In [ ]:
# Assuming we use the most recent season (2023-2024) as the standard for column order
standard_column_order = globals().get('defense_season_2023_2024', pd.DataFrame()).columns.tolist()

# Initialize a list to store the modified DataFrames
all_seasons_data = []

# Loop over the seasons again to align all DataFrames and add the 'season' column
for year in range(17, 24):
    # Construct the variable name
    df_name = f"defense_season_20{year:02d}_20{year+1:02d}"

    # Access the DataFrame using globals()
    if df_name in globals():
        # Create a copy to avoid modifying the original DataFrame
        df = globals()[df_name].copy()

        # Align the DataFrame to the standard column order, filling missing columns with NaN
        df = df.reindex(columns=standard_column_order, fill_value=np.nan)

        # Add the season identifier column
        df['season'] = f"20{year:02d}-20{year+1:02d}"

        # Append the modified DataFrame to the list
        all_seasons_data.append(df)

# Concatenate all DataFrames
defense_lfc = pd.concat(all_seasons_data, ignore_index=True)

In [ ]:
defense_lfc.info()

In [ ]:
defense_lfc['Player'].value_counts()

In [ ]:
defense_lfc = defense_lfc[(defense_lfc['Player'] != 'Squad Total') & (defense_lfc['Player'] != 'Opponent Total')]

In [ ]:
defense_lfc.info()

In [ ]:
defense_lfc.to_csv('defense_lfc.csv', sep=',', index=False, encoding='utf-8-sig')

## Getting possession data for Liverpool FC from EPL Season 2017-2018 to 2023-2024

In [ ]:
for season in fbref_data.keys():
    # Construct the variable name based on the naming convention
    variable_name = f"possession_season_{season.replace('-', '_')}"

    # Assign the DataFrame to the dynamically created variable
    # Using the first table from each season's data
    globals()[variable_name] = fbref_data[season][9]

# Loop over the seasons
for year in range(17, 24):
    # Construct the variable name
    df_name = f"possession_season_20{year:02d}_20{year+1:02d}"

    # Access the DataFrame using globals()
    df = globals()[df_name]

    # Perform the operations
    # 1. Combining multi-level column names
    df.columns = [' '.join(col).strip() for col in df.columns]

    # 2. Resetting the index
    df = df.reset_index(drop=True)

    # 3. Renaming columns to handle 'level_0' occurrences
    new_columns = []
    for col in df.columns:
        if 'level_0' in col:
            new_col = col.split()[-1]  # takes the last name
        else:
            new_col = col
        new_columns.append(new_col)
    df.columns = new_columns

    # 4. Filling NaN values with 0
    df = df.fillna(0)

    # 5. Optionally printing the head (first few rows) of each DataFrame
    print(f"Head of DataFrame for {df_name}:")
    print(df.head())

    # Update the DataFrame in globals()
    globals()[df_name] = df

In [ ]:
# Loop over the seasons and print the shape of each DataFrame
for year in range(17, 24):
    # Construct the variable name based on the naming convention
    df_name = f"possession_season_20{year:02d}_20{year+1:02d}"

    # Access the DataFrame using globals() and print its shape
    if df_name in globals():
        print(f"{df_name}: {globals()[df_name].shape}")
    else:
        print(f"DataFrame {df_name} not found.")

In [ ]:
# Assuming we use the most recent season (2023-2024) as the standard for column order
standard_column_order = globals().get('possession_season_2023_2024', pd.DataFrame()).columns.tolist()

# Initialize a list to store the modified DataFrames
all_seasons_data = []

# Loop over the seasons again to align all DataFrames and add the 'season' column
for year in range(17, 24):
    # Construct the variable name
    df_name = f"possession_season_20{year:02d}_20{year+1:02d}"

    # Access the DataFrame using globals()
    if df_name in globals():
        # Create a copy to avoid modifying the original DataFrame
        df = globals()[df_name].copy()

        # Align the DataFrame to the standard column order, filling missing columns with NaN
        df = df.reindex(columns=standard_column_order, fill_value=np.nan)

        # Add the season identifier column
        df['season'] = f"20{year:02d}-20{year+1:02d}"

        # Append the modified DataFrame to the list
        all_seasons_data.append(df)

# Concatenate all DataFrames
possession_lfc = pd.concat(all_seasons_data, ignore_index=True)

In [ ]:
possession_lfc.info()

In [ ]:
possession_lfc['Player'].value_counts()

In [ ]:
possession_lfc = possession_lfc[(possession_lfc['Player'] != 'Squad Total') & (possession_lfc['Player'] != 'Opponent Total')]

In [ ]:
possession_lfc.info()

In [ ]:
possession_lfc.to_csv('possession_lfc.csv', sep=',', index=False, encoding='utf-8-sig')

## Getting playing time data for Liverpool FC from EPL Season 2017-2018 to 2023-2024

In [ ]:
for season in fbref_data.keys():
    # Construct the variable name based on the naming convention
    variable_name = f"playing_time_season_{season.replace('-', '_')}"

    # Assign the DataFrame to the dynamically created variable
    # Using the first table from each season's data
    globals()[variable_name] = fbref_data[season][10]

# Loop over the seasons
for year in range(17, 24):
    # Construct the variable name
    df_name = f"playing_time_season_20{year:02d}_20{year+1:02d}"

    # Access the DataFrame using globals()
    df = globals()[df_name]

    # Perform the operations
    # 1. Combining multi-level column names
    df.columns = [' '.join(col).strip() for col in df.columns]

    # 2. Resetting the index
    df = df.reset_index(drop=True)

    # 3. Renaming columns to handle 'level_0' occurrences
    new_columns = []
    for col in df.columns:
        if 'level_0' in col:
            new_col = col.split()[-1]  # takes the last name
        else:
            new_col = col
        new_columns.append(new_col)
    df.columns = new_columns

    # 4. Filling NaN values with 0
    df = df.fillna(0)

    # 5. Optionally printing the head (first few rows) of each DataFrame
    print(f"Head of DataFrame for {df_name}:")
    print(df.head())

    # Update the DataFrame in globals()
    globals()[df_name] = df

In [ ]:
# Loop over the seasons and print the shape of each DataFrame
for year in range(17, 24):
    # Construct the variable name based on the naming convention
    df_name = f"playing_time_season_20{year:02d}_20{year+1:02d}"

    # Access the DataFrame using globals() and print its shape
    if df_name in globals():
        print(f"{df_name}: {globals()[df_name].shape}")
    else:
        print(f"DataFrame {df_name} not found.")

In [ ]:
# Assuming we use the most recent season (2023-2024) as the standard for column order
standard_column_order = globals().get('playing_time_season_2023_2024', pd.DataFrame()).columns.tolist()

# Initialize a list to store the modified DataFrames
all_seasons_data = []

# Loop over the seasons again to align all DataFrames and add the 'season' column
for year in range(17, 24):
    # Construct the variable name
    df_name = f"playing_time_season_20{year:02d}_20{year+1:02d}"

    # Access the DataFrame using globals()
    if df_name in globals():
        # Create a copy to avoid modifying the original DataFrame
        df = globals()[df_name].copy()

        # Align the DataFrame to the standard column order, filling missing columns with NaN
        df = df.reindex(columns=standard_column_order, fill_value=np.nan)

        # Add the season identifier column
        df['season'] = f"20{year:02d}-20{year+1:02d}"

        # Append the modified DataFrame to the list
        all_seasons_data.append(df)

# Concatenate all DataFrames
playing_time_lfc = pd.concat(all_seasons_data, ignore_index=True)

In [ ]:
all_seasons_data

In [ ]:
playing_time_lfc.info()

In [ ]:
playing_time_lfc['Player'].value_counts()

In [ ]:
playing_time_lfc = playing_time_lfc[(playing_time_lfc['Player'] != 'Squad Total') & (playing_time_lfc['Player'] != 'Opponent Total')]

In [ ]:
playing_time_lfc.info()

In [ ]:
playing_time_lfc.to_csv('playing_time_lfc.csv', sep=',', index=False, encoding='utf-8-sig')

In [ ]:
playing_time_lfc[playing_time_lfc['Player'] == 'Mohamed Salah']